# Permutation Importance

机器学习通常像一个黑匣子，它们能做出很好的预测，但很难完全理解驱动这些预测的决策。  
从一个模型中获得洞察力不是一件容易的事，尽管这可以帮助我们debug、特征工程、指导未来数据采集、给予人们做决策所需的信息，并最终信任模型的预测。

关于模型的最琐碎的查询之一可能是确定哪些特征对预测的影响最大，称为特征重要性。 评估此指标的一种方法是`permutation importance`。
  
**这种方法的不足**：
1. 计算时间：这个过程在计算上可能很昂贵，因为它需要您遍历每个预测变量并做出预测。 如果做出预测并不便宜，或者如果您有很多很多预测变量，那么这可能会很昂贵。
2. 存在多重共线性时表现不佳：如果您的数据集具有相关特征，则排列特征重要性可能表现不佳。 如果一个预测变量中的信息也存储在相关预测变量中，那么当其中一个预测变量被打乱时，模型可能仍然表现良好。
3. 分数是相对的，而不是绝对的：排列重要性分数显示模型中特征的相对预测能力。 然而，这些分数在脱离上下文的情况下实际上没有任何有意义的价值——任何分数都可能非常好或非常差，具体取决于其他分数。
4. 特征重要性仍然不是统计推断：特征重要性技术只能告诉您预测变量的用处——它们不会深入了解关系的性质（例如线性、二次等）或关系的大小 预测器的作用。 排列特征重要性不是统计推理的替代品，而是无法执行传统推理时的替代解决方案。

-------

一旦在训练集上训练了模型，就会计算`permutation importance`。   
它询问：如果单个属性的数据点随机打乱（在验证集中），将所有剩余数据保持原样，使用这个新数据对准确性有何影响？

理想情况下，列的随机重新排序应该会导致准确性降低，因为新数据与现实世界的统计数据几乎没有关联。   
当模型非常依赖的重要特征被改组时，模型准确性受到的影响最大。 有了这个洞察，过程如下：

1. 获得训练好的模型。
2. 打乱单个属性的值并使用此数据获得新的预测。 接下来，使用这些新值和预测评估损失函数的变化，以确定改组的效果。 性能的下降量化了被洗牌的特征的重要性。
3. 反转上一步中完成的改组以取回原始数据。 使用下一个属性重做步骤 2，直到确定每个特征的重要性。

[示例来源](https://www.geeksforgeeks.org/machine-learning-explainability-using-permutation-importance/)

In [1]:
"""
这里使用Python’s ELI5 library
安装：pip install eli5
"""

from sklearn.datasets import load_boston
  
boston = load_boston()
print(boston.DESCR[20:1420])


Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pupil-teacher ratio by

/Users/qianchunqiang/miniforge3/envs/tf/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the Cal

## 数据集划分

In [2]:
from sklearn.model_selection import train_test_split
  
# separate data into target & independent variables
x = boston.data
y = boston.target
  
# split data into train and test set
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8)
print('Size of: ')
print('Training Set x: ', x_train.shape)
print('Training Set y: ', y_train.shape)
print('Test Set x: ', x_test.shape)
print('Test Set y: ', y_test.shape)

Size of: 
Training Set x:  (404, 13)
Training Set y:  (404,)
Test Set x:  (102, 13)
Test Set y:  (102,)


## 训练模型

In [3]:
from sklearn.ensemble import RandomForestRegressor

# train model on training set
rf = RandomForestRegressor()

# fit model on training set
rf.fit(x_train, y_train)

# calculate score on test set
print('R2 score for test set: ')
print(rf.score(x_test, y_test))

R2 score for test set: 
0.8498636132924253


## 评估特征重要性

In [4]:
import eli5
from eli5.sklearn import PermutationImportance

# create permutation importance object using model
# and fit on test set
perm = PermutationImportance(rf, random_state=1).fit(x_test, y_test)

# display weights using PermutationImportance object
eli5.show_weights(perm, feature_names = boston.feature_names)

Weight,Feature
0.6141 ± 0.2204,LSTAT
0.4871 ± 0.0601,RM
0.0825 ± 0.0180,CRIM
0.0770 ± 0.0097,NOX
0.0428 ± 0.0589,DIS
0.0377 ± 0.0491,PTRATIO
0.0201 ± 0.0111,AGE
0.0070 ± 0.0098,B
0.0062 ± 0.0024,TAX
0.0049 ± 0.0041,INDUS
